# LUME-Impact Basics

In [ ]:
from impact import Impact

In [ ]:
# Nicer plotting
import matplotlib.pyplot as plt
import matplotlib
import os

from bokeh.plotting import output_notebook
from bokeh.plotting import show


from impact.plotting import layout_plot

matplotlib.rcParams["figure.figsize"] = (8, 4)
output_notebook(hide_banner=True)

Point to a valid input file

In [ ]:
ifile = "templates/lcls_injector/ImpactT.in"

Make Impact object

In [ ]:
I = Impact(ifile)

Change some things

In [ ]:
I.header["Np"] = 10000
I.header["Nx"] = 16
I.header["Ny"] = 16
I.header["Nz"] = 16
I.header["Dt"] = 5e-13

# Turn Space Charge off. Both these syntaxes work
I.header["Bcurr"] = 0
I["header:Bcurr"] = 0

# Other switches
I.timeout = None

# Switches for MPI
I.numprocs = 0  # Auto-select

# This is equivalent to:
# I.use_mpi=True
# I.header['Nprow'] = 2
# I.header['Npcol'] = 2

Plot the layout.

In [ ]:
I.plot()

Change stop location. Here this is does the same as `I.ele['stop_1']['s'] = 1.5`.

In [ ]:
I.stop = 1.5

Run Impact-T. This automatically finds the appropriate executable.

In [ ]:
I.run()

Plot now shows the output statistics.

In [ ]:
I.plot()
# plt.savefig('../assets/plot.png', dpi=150)

These are used to create the input.

In [ ]:
I.input.keys()

This is the output parsed

In [ ]:
I.output.keys()

stats from the various fort. files

In [ ]:
I.output["stats"].keys()

Slice info

In [ ]:
I.output["slice_info"].keys()

# Particles

Particles are automatically parsed in to openpmd-beamphysics ParticleGroup objects

In [ ]:
I.output["particles"]

`I.particles` points to this. Get the final particles and calculate some statistics:

In [ ]:
P = I.particles["final_particles"]
P["mean_energy"]

Show the units:

In [ ]:
P.units("mean_energy")

`ParticleGroup` has built-in plotting

In [ ]:
P.plot("delta_z", "pz")
# plt.savefig('../assets/zpz.png', dpi=150)

# Stats

Impact's own calculated statistics can be retieved

In [ ]:
len(I.stat("norm_emit_x")), I.stat("norm_emit_x")[-1]

Stats can also be computed from the particles. For example:

In [ ]:
I.particles["final_particles"]["norm_emit_x"]

Compare these:

In [ ]:
key1 = "mean_z"
key2 = "sigma_x"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="red",
)

This kind of plot is built-in for convenience, with a layout:

In [ ]:
I.plot("sigma_x")

Even fancier options, and sending some options to matplotlib:

In [ ]:
I.plot(
    ["sigma_x", "sigma_y"],
    y2=["mean_kinetic_energy"],
    ylim2=(0, 8e6),
    figsize=(10, 5),
    include_field=True,
)

# Partial tracking

Particles can be started anywhere in the lattice. Here we will take some intermediate particles, and re-track. 

Get particles at the `YAG02` marker:

In [ ]:
Pmid = I.particles["YAG02"]

Make a copy, so that the previous object is preserved. 

In [ ]:
I2 = I.copy()
I.verbose = False

The copy needs to be configured before tracking

In [ ]:
I2.configure()

Track to 2 m

In [ ]:
Pfinal = I2.track(Pmid, 2.0)

Compare these:

In [ ]:
key1 = "mean_z"
key2 = "sigma_x"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2))
plt.plot(I2.stat(key1), I2.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="red",
)

# Blue X are retracked particles
plt.scatter(
    [P[key1] for P in [Pmid, Pfinal]],
    [P[key2] for P in [Pmid, Pfinal]],
    color="blue",
    marker="x",
)

# Single particle tracking

Similar to above, but with initial conditions specified in the function for a single particle. 

This is useful for auto-phasing and scaling elements, and tracing reference orbits. 

Space charge is turned off for single particle tracking.

In [ ]:
%%time
I3 = I.copy()
I3.verbose = False
I3.configure()
P3 = I3.track1(s=2.2, z0=1.0, pz0=10e6)
P3.z, P3.gamma

# Interactive Layout

Plots can be made interctive via [bokeh](https://docs.bokeh.org/en/latest/#)

Change something and plot:

In [ ]:
I.ele["QE01"]["b1_gradient"] = 0
layout = layout_plot(I.input["lattice"], height=300)
show(layout)

# ControlGroup objects

Some elements need to be changed together, either relatively or absolutely. A single traveling wave cavity, for example, is made from four fieldmaps, with defined relative phases


In [ ]:
for name in ["L0A_entrance", "L0A_body_1", "L0A_body_2", "L0A_exit"]:
    print(name, I[name]["theta0_deg"])

Make a copy and add a group to control these. 

In [ ]:
I4 = I.copy()
I4.add_group(
    "L0A",
    ele_names=["L0A_entrance", "L0A_body_1", "L0A_body_2", "L0A_exit"],
    var_name="theta0_deg",
    attributes="theta0_deg",
)

Make a change

In [ ]:
I4["L0A"]["theta0_deg"] = 0.123456

These get propagated to the underlying elements

In [ ]:
for name in I4["L0A"].ele_names:
    print(name, I4[name]["theta0_deg"])

Set overall scaling, respecting the special factors. 

In [ ]:
I4.add_group(
    "L0A_scale",
    ele_names=["L0A_entrance", "L0A_body_1", "L0A_body_2", "L0A_exit"],
    var_name="rf_field_scale",
    factors=[0.86571945106805, 1, 1, 0.86571945106805],  # sin(k*d) with d = 3.5e-2 m
    absolute=True,
)

I4["L0A_scale"]["rf_field_scale"] = 10

These get propagated to the underlying elements

In [ ]:
for name in I4["L0A_scale"].ele_names:
    print(name, I4[name]["rf_field_scale"])

# Instantiate from YAML

All of the Impact object init arguments can be passed in a YAML file. Any of:

In [ ]:
?Impact

In [ ]:
YAML = """

# Any argument above. One exception is initial_particles: this should be a filename that is parsed into a ParticleGroup

input_file: templates/lcls_injector/ImpactT.in

verbose: False

group:
  L0A:
    ele_names: [ L0A_entrance, L0A_body_1, L0A_body_2, L0A_exit ]
    var_name: dtheta0_deg
    attributes: theta0_deg
    value: 0
    
  L0B:
    ele_names: [ L0B_entrance, L0B_body_1, L0B_body_2, L0B_exit ]
    var_name: dtheta0_deg
    attributes: theta0_deg
    value: 0    
    
  L0A_scale:
    ele_names:  [ L0A_entrance, L0A_body_1, L0A_body_2, L0A_exit ]
    var_name: rf_field_scale
    factors: [0.86571945106805, 1, 1, 0.86571945106805]  # sin(k*d) with d = 3.5e-2 m 
    absolute: True 
    value: 60e6
    
  L0B_scale:
    ele_names:  [ L0B_entrance, L0B_body_1, L0B_body_2, L0B_exit ]
    var_name: rf_field_scale
    factors: [0.86571945106805, 1, 1, 0.86571945106805]  # sin(k*d) with d = 3.5e-2 m 
    absolute: True
    value: 60.0e6
    

"""
I5 = Impact.from_yaml(YAML)
I5["L0A:dtheta0_deg"], I5["L0A_entrance:theta0_deg"]

In [ ]:
I5["L0A"].reference_values

In [ ]:
I5["L0A"]

# Autophase

Autophase will calculate the relative phases of each rf element by tracking a single particle through the fieldmaps. This is done externally to Impact, and is relatively fast.

A call to `Impact.autophase()` returns the relative phases found as a dict:

In [ ]:
I5.autophase()

You can also give it a dict of `ele_name:rel_phase_deg` with relative phases in degrees, and it will set these as it phases:

In [ ]:
I5.autophase({"GUN": -9, "L0A": 2})

# Archive all output

All of .input and .output can be archived and loaded from standard h5 files.

Particles are stored in the openPMD-beamphysics format.

Call the `archive` method. If no name is given, a name will be invented based on the fingerprint.

In [ ]:
afile = I.archive()

This can be loaded into an empty model

In [ ]:
I2 = Impact()
I2.load_archive(afile)

This also works:

In [ ]:
I2 = Impact.from_archive(afile)

Check that the fingerprints are the same

In [ ]:
assert I.fingerprint() == I2.fingerprint()

Look at a stat, and compare with the original object

In [ ]:
I.stat("norm_emit_x")[-1], I2.stat("norm_emit_x")[-1]

The particles look the same:

In [ ]:
I2.particles["final_particles"].plot("delta_z", "pz")

# Cleanup

In [ ]:
os.remove(afile)